In [1]:

import gc
import pandas as pd
import numpy as np
import sklearn as skl
import seaborn as sns
import torch
import scipy
import opendatasets as od
import math
import torchmetrics
import importlib
import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import category_encoders

import utils.mlp as mlp
import utils.mlp_pipeline as mlp_pipeline
import utils.lgbm as lgbm

from sklearn.preprocessing import LabelEncoder 
import yaml

from bank_account_fraud.notebooks.random_search import RandomValueTrial, suggest_callable_hyperparams  # from https://github.com/feedzai/bank-account-fraud.git



In [2]:
import pickle
import io

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)
class RenameUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        renamed_module = module
        if module == "gdrive.MyDrive.rtdl.rtdl.data":
            renamed_module = "rtdl.rtdl.data"

        return super(RenameUnpickler, self).find_class(renamed_module, name)


def renamed_load(file_obj):
    return RenameUnpickler(file_obj).load()


def renamed_loads(pickled_bytes):
    file_obj = io.BytesIO(pickled_bytes)
    return renamed_load(file_obj)

In [3]:
def find_optimal_recall(fpr,tpr,threshold,rate):
    optimal_index=np.argmin(abs(fpr-rate*np.ones(len(fpr))))
    print(fpr[optimal_index])
    print(threshold[optimal_index])
    return tpr[optimal_index]

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")



Using cpu device


In [5]:
#Kaggle api key: 9c2fc93aaaf8815d9fa8f2ceeeb57b6b
od.download("https://www.kaggle.com/datasets/sgpjesus/bank-account-fraud-dataset-neurips-2022?select=Base.csv")

Skipping, found downloaded files in ".\bank-account-fraud-dataset-neurips-2022" (use force=True to force download)


In [5]:
import glob

extension = "csv"  # or "parquet", depending on the downloaded file
data_paths = glob.glob(f"</path/to/datasets/>*.{extension}")

def read_dataset(path, ext=extension):
    if ext == "csv":
        return pd.read_csv(path, index_col=0)
    elif ext == "parquet":
        return pd.read_parquet(path)
    else:
        raise ValueError(f"Invalid extension: '{ext}'.")

def get_variant(path):
        return path.split("/")[-1].split(".")[0]

dataframes = {
    get_variant(path): read_dataset(path) for path in data_paths
}

In [6]:


# Read hyperparameter space for the LGBM Models, expected structure is presented bellow
with open("bank_account_fraud/notebooks/lightgbm_hyperparameter_space.yaml", "r") as file:
    hyperparam_space = yaml.load(file, Loader=yaml.FullLoader)



In [7]:


# Define path to datasets. Replace `base_path` with the appropriate value.
base_path = "bank-account-fraud-dataset-neurips-2022/"

datasets_paths = {
    "Base":    base_path + "Base.csv",
}

datasets = {key: pd.read_csv(path) for key, path in datasets_paths.items()}

In [ ]:
print(datasets['Base'].describe())


# Create the train and test sets. Shuffle data with `sample` method.
# The split was done by month. The first 6 months as the train, the last 2 months as test.
train_dfs = {key: df[df["month"]<6].sample(frac=1, replace=False) for key, df in datasets.items()}
val_dfs = {key: df[df["month"]==6].sample(frac=1, replace=False) for key, df in datasets.items()}
test_dfs= {key: df[df["month"]==7].sample(frac=1, replace=False) for key, df in datasets.items()}

label = "fraud_bool"

categorical_features = [
    "payment_type",
    "employment_status",
    "housing_status",
    "source",
    "device_os",
]

In [9]:

for name in datasets.keys():  # For each dataset in the suite
    train = train_dfs[name]
    val=val_dfs[name]
    test = test_dfs[name]
    for feat in categorical_features:
        encoder = LabelEncoder()
        encoder.fit(train[feat])  # Fit an encoder to the train set.
        train[feat] = encoder.transform(train[feat])  # Transform train set.
        val[feat] = encoder.transform(val[feat])  # Transform val set.
        test[feat] = encoder.transform(test[feat])    # Transform test set.

In [10]:
for dataset_name in datasets.keys():  # Run hyperparameters on all variants of datastes.
        
    X_train = train_dfs[dataset_name].drop(columns=["fraud_bool"])
    y_train = train_dfs[dataset_name]["fraud_bool"]
    X_val = val_dfs[dataset_name].drop(columns=["fraud_bool"])
    y_val = val_dfs[dataset_name]["fraud_bool"]
    X_test=test_dfs[dataset_name].drop(columns=["fraud_bool"])
    y_test = test_dfs[dataset_name]["fraud_bool"]

In [11]:
#Establish continuous and categorical features
cat_cols=categorical_features
cont_cols=train.columns.difference(cat_cols)
cont_cols=cont_cols.difference(['fraud_bool'])

link="tab_norm_folder-20230130/tab_norm_folder/encoders/numerical_encoder_dim10.pkl"
file = open(link,'rb')
encoder =renamed_load(file)
transformed_val=encoder.transform(torch.tensor(X_val[cont_cols.difference(['device_fraud_count'])].values))
transformed_train=encoder.transform(torch.tensor(X_train[cont_cols.difference(['device_fraud_count'])].values))
X_val_aux=torch.cat([torch.tensor(X_val[cat_cols].values),transformed_val],1).float()
X_train_aux=torch.cat([torch.tensor(X_train[cat_cols].values),transformed_train],1).float()


In [ ]:


# Define number of trials in Random search.
n_trials=20

param_list=mlp.mlp_param_sampler(n_trials, len(train.columns),7,device)

In [ ]:
i=0

for normalization in ['None','Zscore','MinMax','Median']
    xtrain_aux=mlp_pipeline.normalization_transform(X_train,normalization,cont_cols)
    xval_aux=mlp_pipeline.normalization_transform(X_val,normalization,cont_cols)

    for params in param_list[i:]:

        # Fit pipeline
        print('iteration: ',i)
        
        model=mlp_pipeline.pipeline(device,xtrain_aux,xval_aux,y_train,y_val,params,'None',cont_cols)

        #Save the model
        joblib.dump(model,'/content/gdrive/MyDrive/tab_norm_folder/mlp_simple{}{}.pkl'.format(normalization,i))

        i=i+1
    del xval_aux       
    del xtrain_aux
    gc.collect()



## **Encodings**

In [14]:
target_encoder=category_encoders.target_encoder.TargetEncoder(cols=cat_cols,handle_unknown='value',handle_missing='value')
xtrain=target_encoder.fit_transform(X_train,y_train)
xval=target_encoder.transform(X_val)

C:\Users\ritas\anaconda3\lib\site-packages\category_encoders\target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\ritas\anaconda3\lib\site-packages\category_encoders\target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [ ]:

# Define number of trials in Random search.
n_trials=20

param_list=mlp.mlp_param_sampler(n_trials, len(train.columns),7,device)


for normalization in ['Zscore','None','MinMax','Median']:

    xtrain_aux=mlp_pipeline.normalization_transform(xtrain,normalization,train.columns.difference(['fraud_bool']))
    xval_aux=mlp_pipeline.normalization_transform(xval,normalization,train.columns.difference(['fraud_bool']))
    i=0

    for params in param_list[i:]:

        # Fit pipeline
        print('iteration: ',i)
        
        model=mlp_pipeline.pipeline(device,xtrain_aux,xval_aux,y_train,y_val,params,'None',train.columns.difference(['fraud_bool']))

        #Save the model
        joblib.dump(model,'/content/gdrive/MyDrive/tab_norm_folder/mlp_target{}{}.pkl'.format(normalization,i))

        i=i+1
    del xval_aux
    del xtrain_aux
    gc.collect()

In [15]:
import category_encoders
catboost_encoder=category_encoders.cat_boost.CatBoostEncoder(cols=cat_cols,handle_unknown='value',handle_missing='value',drop_invariant=False,return_df=True,sigma=0)
xtrain_cat=catboost_encoder.fit_transform(X_train,y_train)
xval_cat=catboost_encoder.transform(X_val)

In [ ]:
import mlp
import mlp_pipeline
mlp_pipeline=importlib.reload(mlp_pipeline)

# Define number of trials in Random search.
n_trials=20

param_list=mlp.mlp_param_sampler(n_trials, len(train.columns),7,device)
i=0


for normalization in ['Zscore','None','MinMax','Median']:
    xtrain_aux=mlp_pipeline.normalization_transform(xtrain_cat,normalization,train.columns.difference(['fraud_bool']))
    xval_aux=mlp_pipeline.normalization_transform(xval_cat,normalization,train.columns.difference(['fraud_bool']))
    i=0
    for params in param_list[i:]:

        # Fit pipeline
        print('iteration: ',i)
        
        model=mlp_pipeline.pipeline(device,xtrain_aux,xval_aux,y_train,y_val,params,'None',train.columns.difference(['fraud_bool']))

        #Save the model
        joblib.dump(model,'/content/gdrive/MyDrive/tab_norm_folder/mlp_cat{}{}.pkl'.format(normalization,i))

        i=i+1
    del xval_aux
    del xtrain_aux

    gc.collect()

In [14]:
count_encoder=category_encoders.count.CountEncoder(cols=cat_cols,verbose=0,handle_unknown='value',handle_missing='value',normalize=True,min_group_size=0.005,min_group_name='Leftover',combine_min_nan_groups=False)
xtrain_cont=count_encoder.fit_transform(X_train,y_train)
xval_cont=count_encoder.transform(X_val)

In [ ]:
import mlp
import mlp_pipeline
mlp_pipeline=importlib.reload(mlp_pipeline)

# Define number of trials in Random search.
n_trials=20
# Random state for sampling seeds.
np.random.seed(42)

# Variable to store the results.
runs = {}
param_list=mlp.mlp_param_sampler(n_trials, len(train.columns),7,device)

for normalization in ['None','Zscore','MinMax','Median']:
    xtrain_aux=mlp_pipeline.normalization_transform(xtrain_cont,normalization,train.columns.difference(['fraud_bool']))
    xval_aux=mlp_pipeline.normalization_transform(xval_cont,normalization,train.columns.difference(['fraud_bool']))
    i=0
    for params in param_list[i:]:

        # Fit pipeline
        print('iteration: ',i)
        
        model=mlp_pipeline.pipeline(device,xtrain_aux,xval_aux,y_train,y_val,params,'None',train.columns.difference(['fraud_bool']))

        #Save the model
        joblib.dump(model,'/content/gdrive/MyDrive/tab_norm_folder/mlp_count{}{}.pkl'.format(normalization,i))

        i=i+1
    del xval_aux  
    del xtrain_aux  
    gc.collect()

In [16]:
joblib.dump(count_encoder,'tab_norm_folder/count_encoderBAF.pkl')
joblib.dump(target_encoder,'tab_norm_folder/target_encoderBAF.pkl')
joblib.dump(catboost_encoder,'tab_norm_folder/catboost_encoderBAF.pkl')

['/content/gdrive/MyDrive/tab_norm_folder/count_encoderBAF.pkl']

In [13]:
param_list=lgbm.lgbm_param_sampler(20,7,'cpu')

In [ ]:
for encoding in ['cat','target','count']:
    encoder_link="tab_norm_folder-20230202/tab_norm_folder/{}_encoderBAF.pkl".format(encoding)
    encoder = joblib.load(encoder_link)
    xtrain=encoder.transform(X_train)
    xval=encoder.transform(X_val)
    X_val_aux=torch.cat([torch.tensor(xval[cat_cols].values),transformed_val],1).float()
    X_train_aux=torch.cat([torch.tensor(xtrain[cat_cols].values),transformed_train],1).float()

    

    i=0
    n_trials=20
    seed=7

    i=0
    for trial in range(n_trials):
 
        test_hyperparams = lgbm.lgbm_param_sampler(20, seed,device)    
    
        model = lgb.LGBMClassifier(n_jobs=10, **test_hyperparams)  
        
        # Fit model to training data.
        model.fit(X_train_aux, y_train, categorical_feature=[],eval_set=[(X_val_aux,y_val)],early_stopping_rounds=5,eval_metric='loss')
        # Obtain predictions in test data.
        predictions = model.predict_proba(X_val_aux)[:, 1]
        print(skl.metrics.roc_auc_score(y_val,predictions))
        joblib.dump(model,'LGBM_BAF/lgbm_numericalemb_{}{}.pkl'.format(encoding,i))
        i=i+1

In [ ]:

i=0
n_trials=20
seeds = np.random.choice(list(range(1_000_000)), size=n_trials, replace=False)

i=0
for trial in range(n_trials):
    test_hyperparams = lgbm.lgbm_param_sampler(20, seed,device)    
    
    model = lgb.LGBMClassifier(n_jobs=10, **test_hyperparams)  
        
    model.fit(X_train_aux, y_train, categorical_feature=[0,1,2,3,4],eval_set=[(X_val_aux,y_val)],early_stopping_rounds=5,eval_metric='loss')
    predictions = model.predict_proba(X_val_aux)[:, 1]
    print(skl.metrics.roc_auc_score(y_val,predictions))
    joblib.dump(model,'LGBM_BAF/lgbm_numericalemb_label{}.pkl'.format(i))
    i=i+1

In [ ]:
for encoding in ['count']:
    encoder_link="tab_norm_folder-20230202/tab_norm_folder/{}_encoderBAF.pkl".format(encoding)
    encoder = joblib.load(encoder_link)
    xtrain=encoder.transform(X_train)
    xval=encoder.transform(X_val)
    X_val_aux=torch.cat([torch.tensor(xval[cat_cols].values),transformed_val],1).float()
    X_train_aux=torch.cat([torch.tensor(xtrain[cat_cols].values),transformed_train],1).float()

    

    i=0
    n_trials=20

    i=0
    for trial in range(n_trials):
        seed = seeds[trial]
        trial = RandomValueTrial(seed=seed)

        test_hyperparams = lgbm.lgbm_param_sampler(20, seed,device)    
    

        model = lgb.LGBMClassifier(n_jobs=10, **test_hyperparams)  # Instantiate LGBM Model.
        
        # Fit model to training data.
        model.fit(X_train_aux, y_train, categorical_feature=[],eval_set=[(X_val_aux,y_val)],early_stopping_rounds=5,eval_metric='loss')
        # Obtain predictions in test data.
        predictions = model.predict_proba(X_val_aux)[:, 1]
        print(skl.metrics.roc_auc_score(y_val,predictions))
        joblib.dump(model,'LGBM_BAF/lgbm_numericalemb_{}{}.pkl'.format(encoding,i))
        i=i+1

In [13]:
for encoding in ['count']:
    encoder_link="tab_norm_folder-20230202/tab_norm_folder/{}_encoderBAF.pkl".format(encoding)
    encoder = joblib.load(encoder_link)
    xtrain=encoder.transform(X_train)
    xval=encoder.transform(X_val)
    print(xtrain)
    i=0
    n_trials=20
   
    runs = {}
    i=0
    for trial in range(n_trials):
        test_hyperparams = lgbm.lgbm_param_sampler(20, seed,device)    
    
        model = lgb.LGBMClassifier(n_jobs=10, **test_hyperparams) 
        
        # Fit model to training data.
        model.fit(xtrain, y_train, categorical_feature=[],eval_set=[(xval,y_val)],early_stopping_rounds=5,eval_metric='loss')
        # Obtain predictions in test data.
        predictions = model.predict_proba(xval)[:, 1]
        print(skl.metrics.roc_auc_score(y_val,predictions))
        joblib.dump(model,'LGBM_BAF/lgbm_simple_{}{}.pkl'.format(encoding,i))
        i=i+1

        income  name_email_similarity  prev_address_months_count  \
966106     0.7               0.839614                         -1   
210362     0.4               0.031270                         31   
200155     0.9               0.596591                         -1   
998704     0.9               0.908534                         -1   
891845     0.7               0.084132                         -1   
...        ...                    ...                        ...   
486902     0.6               0.438007                         99   
952611     0.9               0.839012                         62   
263448     0.6               0.834232                         26   
506400     0.3               0.203240                         -1   
864226     0.1               0.916049                         -1   

        current_address_months_count  customer_age  days_since_request  \
966106                            55            40            0.022193   
210362                            1

C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_data_in_leaf is set=22, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=22
[1]	valid_0's binary_logloss: 0.0681796
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.0662972
[3]	valid_0's binary_logloss: 0.0648805
[4]	valid_0's binary_logloss: 0.063738
[5]	valid_0's binary_logloss: 0.0628749
[6]	valid_0's binary_logloss: 0.0620757
[7]	valid_0's binary_logloss: 0.0614054
[8]	valid_0's binary_logloss: 0.0608747
[9]	valid_0's binary_logloss: 0.0604189
[10]	valid_0's binary_logloss: 0.0600095
[11]	valid_0's binary_logloss: 0.0595799
[12]	valid_0's binary_logloss: 0.059206
[13]	valid_0's binary_logloss: 0.0588104
[14]	valid_0's binary_logloss: 0.058501
[15]	valid_0's binary_logloss: 0.0582398
[16]	valid_0's binary_logloss: 0.0579698
[17]	valid_0's binary_logloss: 0.0577499
[18]	valid_0's binary_logloss: 0.057518
[19]	valid_0's binary_logloss: 0.0572936
[20]	valid_0's binary_logloss: 0.0571262
[21]	valid_

C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_data_in_leaf is set=9, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9
[1]	valid_0's binary_logloss: 0.0688725
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.0674091
[3]	valid_0's binary_logloss: 0.0661999
[4]	valid_0's binary_logloss: 0.0651382
[5]	valid_0's binary_logloss: 0.0642341
[6]	valid_0's binary_logloss: 0.0634459
[7]	valid_0's binary_logloss: 0.0628868
[8]	valid_0's binary_logloss: 0.0623342
[9]	valid_0's binary_logloss: 0.0618805
[10]	valid_0's binary_logloss: 0.0614895
[11]	valid_0's binary_logloss: 0.0611052
[12]	valid_0's binary_logloss: 0.0607243
[13]	valid_0's binary_logloss: 0.0603548
[14]	valid_0's binary_logloss: 0.0600709
[15]	valid_0's binary_logloss: 0.0597127
[16]	valid_0's binary_logloss: 0.0594559
[17]	valid_0's binary_logloss: 0.0591522
[18]	valid_0's binary_logloss: 0.0589122
[19]	valid_0's binary_logloss: 0.0586855
[20]	valid_0's binary_logloss: 0.0584363
Did not m

C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[1]	valid_0's binary_logloss: 0.0667247
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.0650373
[3]	valid_0's binary_logloss: 0.0636238
[4]	valid_0's binary_logloss: 0.0626437
[5]	valid_0's binary_logloss: 0.0617207
[6]	valid_0's binary_logloss: 0.0610797
[7]	valid_0's binary_logloss: 0.0604793
[8]	valid_0's binary_logloss: 0.0599358
[9]	valid_0's binary_logloss: 0.0594966
[10]	valid_0's binary_logloss: 0.0590811
[11]	valid_0's binary_logloss: 0.0586962
[12]	valid_0's binary_logloss: 0.0583889
[13]	valid_0's binary_logloss: 0.0581161
[14]	valid_0's binary_logloss: 0.057939
[15]	valid_0's binary_logloss: 0.0577033
[16]	valid_0's binary_logloss: 0.057519
[17]	valid_0's binary_logloss: 0.0573613
[18]	valid_0's binary_logloss: 0.0571607
[19]	valid_0's binary_logloss: 0.0570428
[20]	valid_0's binary_logloss: 0.0568744
[21]	valid_

KeyboardInterrupt: 

In [12]:
i=0
n_trials=20

i=0
for trial in range(n_trials):
    test_hyperparams = lgbm.lgbm_param_sampler(20, seed,device)    
    
    model = lgb.LGBMClassifier(n_jobs=10, **test_hyperparams)  # Instantiate LGBM Model.
        
    # Fit model to training data.
    model.fit(X_train[cont_cols], y_train, categorical_feature=[],eval_set=[(X_val[cont_cols],y_val)],early_stopping_rounds=5,eval_metric='loss')
    # Obtain predictions in test data.
    predictions = model.predict_proba(X_test[cont_cols])[:, 1]
    print(skl.metrics.roc_auc_score(y_test,predictions))
    joblib.dump(model,'LGBM_BAF/lgbm_onlycontfeatures{}.pkl'.format(i))
    i=i+1

C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_data_in_leaf is set=22, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=22
[1]	valid_0's binary_logloss: 0.0689625
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.0675481
[3]	valid_0's binary_logloss: 0.0664684
[4]	valid_0's binary_logloss: 0.065648
[5]	valid_0's binary_logloss: 0.0648556
[6]	valid_0's binary_logloss: 0.0641481
[7]	valid_0's binary_logloss: 0.0635689
[8]	valid_0's binary_logloss: 0.0630028
[9]	valid_0's binary_logloss: 0.0625314
[10]	valid_0's binary_logloss: 0.0620863
[11]	valid_0's binary_logloss: 0.0616778
[12]	valid_0's binary_logloss: 0.0613016
[13]	valid_0's binary_logloss: 0.0610052
[14]	valid_0's binary_logloss: 0.0607285
[15]	valid_0's binary_logloss: 0.0604766
[16]	valid_0's binary_logloss: 0.0602522
[17]	valid_0's binary_logloss: 0.0600341
[18]	valid_0's binary_logloss: 0.059843
[19]	valid_0's binary_logloss: 0.0596309
[20]	valid_0's binary_logloss: 0.0594596
[21]	vali

C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_data_in_leaf is set=9, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9
[1]	valid_0's binary_logloss: 0.0697082
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.068645
[3]	valid_0's binary_logloss: 0.0677445
[4]	valid_0's binary_logloss: 0.0670125
[5]	valid_0's binary_logloss: 0.0663462
[6]	valid_0's binary_logloss: 0.065557
[7]	valid_0's binary_logloss: 0.0650413
[8]	valid_0's binary_logloss: 0.064584
[9]	valid_0's binary_logloss: 0.0640936
[10]	valid_0's binary_logloss: 0.0635811
[11]	valid_0's binary_logloss: 0.0631715
[12]	valid_0's binary_logloss: 0.0628049
[13]	valid_0's binary_logloss: 0.0624477
[14]	valid_0's binary_logloss: 0.0621029
[15]	valid_0's binary_logloss: 0.0617965
[16]	valid_0's binary_logloss: 0.0615311
[17]	valid_0's binary_logloss: 0.0612607
[18]	valid_0's binary_logloss: 0.0610423
[19]	valid_0's binary_logloss: 0.0608344
[20]	valid_0's binary_logloss: 0.0606138
Did not meet

C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


...

[LightGBM] [Warning] min_data_in_leaf is set=66, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=66
[1]	valid_0's binary_logloss: 0.0704309
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.0697233
[3]	valid_0's binary_logloss: 0.0690808
[4]	valid_0's binary_logloss: 0.068533
[5]	valid_0's binary_logloss: 0.0680991
[6]	valid_0's binary_logloss: 0.0676453
[7]	valid_0's binary_logloss: 0.0671085
[8]	valid_0's binary_logloss: 0.0666763
[9]	valid_0's binary_logloss: 0.0662977
[10]	valid_0's binary_logloss: 0.0659013
[11]	valid_0's binary_logloss: 0.0656143
[12]	valid_0's binary_logloss: 0.0653167
[13]	valid_0's binary_logloss: 0.0649876
[14]	valid_0's binary_logloss: 0.0646871
[15]	valid_0's binary_logloss: 0.0643881
[16]	valid_0's binary_logloss: 0.0641281
[17]	valid_0's binary_logloss: 0.0638743
[18]	valid_0's binary_logloss: 0.063626
[19]	valid_0's binary_logloss: 0.0633955
[20]	valid_0's binary_logloss: 0.0631534
[21]	vali

[195]	valid_0's binary_logloss: 0.0561579
[196]	valid_0's binary_logloss: 0.056152
[197]	valid_0's binary_logloss: 0.0561459
[198]	valid_0's binary_logloss: 0.0561421
[199]	valid_0's binary_logloss: 0.0561344
[200]	valid_0's binary_logloss: 0.0561291
[201]	valid_0's binary_logloss: 0.0561268
[202]	valid_0's binary_logloss: 0.0561252
[203]	valid_0's binary_logloss: 0.0561189
[204]	valid_0's binary_logloss: 0.0561111
[205]	valid_0's binary_logloss: 0.0561092
[206]	valid_0's binary_logloss: 0.0561004
[207]	valid_0's binary_logloss: 0.0561041
[208]	valid_0's binary_logloss: 0.0560941
[209]	valid_0's binary_logloss: 0.0560867
[210]	valid_0's binary_logloss: 0.0560839
[211]	valid_0's binary_logloss: 0.0560783
[212]	valid_0's binary_logloss: 0.0560749
[213]	valid_0's binary_logloss: 0.0560723
[214]	valid_0's binary_logloss: 0.0560668
[215]	valid_0's binary_logloss: 0.0560638
[216]	valid_0's binary_logloss: 0.0560618
[217]	valid_0's binary_logloss: 0.0560637
[218]	valid_0's binary_logloss: 0.0

C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_data_in_leaf is set=33, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=33
[1]	valid_0's binary_logloss: 0.0707577
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.0701952
[3]	valid_0's binary_logloss: 0.0697149
[4]	valid_0's binary_logloss: 0.0693193
[5]	valid_0's binary_logloss: 0.0688912
[6]	valid_0's binary_logloss: 0.0684778
[7]	valid_0's binary_logloss: 0.0681019
[8]	valid_0's binary_logloss: 0.0677665
[9]	valid_0's binary_logloss: 0.0673823
[10]	valid_0's binary_logloss: 0.06706
[11]	valid_0's binary_logloss: 0.0667855
[12]	valid_0's binary_logloss: 0.0665006
[13]	valid_0's binary_logloss: 0.066192
[14]	valid_0's binary_logloss: 0.0659711
[15]	valid_0's binary_logloss: 0.065674
[16]	valid_0's binary_logloss: 0.0654555
[17]	valid_0's binary_logloss: 0.065258
[18]	valid_0's binary_logloss: 0.0649833
[19]	valid_0's binary_logloss: 0.0647497
[20]	valid_0's binary_logloss: 0.064521
[21]	valid_0'

[197]	valid_0's binary_logloss: 0.056524
[198]	valid_0's binary_logloss: 0.0565205
[199]	valid_0's binary_logloss: 0.056516
[200]	valid_0's binary_logloss: 0.056507
[201]	valid_0's binary_logloss: 0.0564991
[202]	valid_0's binary_logloss: 0.0564859
[203]	valid_0's binary_logloss: 0.0564789
[204]	valid_0's binary_logloss: 0.0564681
[205]	valid_0's binary_logloss: 0.0564626
[206]	valid_0's binary_logloss: 0.0564539
[207]	valid_0's binary_logloss: 0.0564466
[208]	valid_0's binary_logloss: 0.0564347
[209]	valid_0's binary_logloss: 0.0564255
[210]	valid_0's binary_logloss: 0.0564168
[211]	valid_0's binary_logloss: 0.0564127
[212]	valid_0's binary_logloss: 0.0564066
[213]	valid_0's binary_logloss: 0.0563991
[214]	valid_0's binary_logloss: 0.0563923
[215]	valid_0's binary_logloss: 0.0563871
[216]	valid_0's binary_logloss: 0.0563833
[217]	valid_0's binary_logloss: 0.0563799
[218]	valid_0's binary_logloss: 0.0563751
[219]	valid_0's binary_logloss: 0.0563706
[220]	valid_0's binary_logloss: 0.056

In [17]:
i=0
for trial in range(n_trials):
    test_hyperparams = lgbm.lgbm_param_sampler(20, seed,device)    
    
    model = lgb.LGBMClassifier(n_jobs=10, **test_hyperparams)  # Instantiate LGBM Model.
        
    # Fit model to training data.
    model.fit(X_train, y_train, categorical_feature=cat_cols,eval_set=[(X_val,y_val)],early_stopping_rounds=5,eval_metric='loss')
    # Obtain predictions in test data.
    predictions = model.predict_proba(X_val)[:, 1]
    print(skl.metrics.roc_auc_score(y_val,predictions))
    joblib.dump(model,'LGBM_BAF/lgbm_{}.pkl'.format(i))
    i=i+1

C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['device_os', 'employment_status', 'housing_status', 'payment_type', 'source']
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_data_in_leaf is set=22, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=22


C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[1]	valid_0's binary_logloss: 0.067975
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.066039
[3]	valid_0's binary_logloss: 0.064655
[4]	valid_0's binary_logloss: 0.0635447
[5]	valid_0's binary_logloss: 0.0626422
[6]	valid_0's binary_logloss: 0.0618544
[7]	valid_0's binary_logloss: 0.0611558
[8]	valid_0's binary_logloss: 0.0606248
[9]	valid_0's binary_logloss: 0.0601353
[10]	valid_0's binary_logloss: 0.0597253
[11]	valid_0's binary_logloss: 0.0593351
[12]	valid_0's binary_logloss: 0.0589677
[13]	valid_0's binary_logloss: 0.0586229
[14]	valid_0's binary_logloss: 0.0583427
[15]	valid_0's binary_logloss: 0.0579949
[16]	valid_0's binary_logloss: 0.057724
[17]	valid_0's binary_logloss: 0.0574073
[18]	valid_0's binary_logloss: 0.0571663
[19]	valid_0's binary_logloss: 0.0569564
[20]	valid_0's binary_logloss: 0.0567964
[21]	valid_0's binary_logloss: 0.0565846
[22]	valid_0's binary_logloss: 0.0564154
[23]	valid_0's binary_logloss: 0.05626
[24]	valid_0

C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['device_os', 'employment_status', 'housing_status', 'payment_type', 'source']
  _log_warning('categorical_feature in Dataset is overridden.\n'


...

C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['device_os', 'employment_status', 'housing_status', 'payment_type', 'source']
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_data_in_leaf is set=33, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=33


C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\ritas\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[1]	valid_0's binary_logloss: 0.0703528
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.0693925
[3]	valid_0's binary_logloss: 0.0686152
[4]	valid_0's binary_logloss: 0.0679974
[5]	valid_0's binary_logloss: 0.067451
[6]	valid_0's binary_logloss: 0.0669157
[7]	valid_0's binary_logloss: 0.0664414
[8]	valid_0's binary_logloss: 0.0659474
[9]	valid_0's binary_logloss: 0.0655513
[10]	valid_0's binary_logloss: 0.0651026
[11]	valid_0's binary_logloss: 0.0647686
[12]	valid_0's binary_logloss: 0.0644091
[13]	valid_0's binary_logloss: 0.0640946
[14]	valid_0's binary_logloss: 0.063786
[15]	valid_0's binary_logloss: 0.0635345
[16]	valid_0's binary_logloss: 0.0632667
[17]	valid_0's binary_logloss: 0.0630217
[18]	valid_0's binary_logloss: 0.0628029
[19]	valid_0's binary_logloss: 0.062557
[20]	valid_0's binary_logloss: 0.0623568
[21]	valid_0's binary_logloss: 0.0621549
[22]	valid_0's binary_logloss: 0.0619512
[23]	valid_0's binary_logloss: 0.0617858
[24]	vali

[201]	valid_0's binary_logloss: 0.0537269
[202]	valid_0's binary_logloss: 0.053717
[203]	valid_0's binary_logloss: 0.053705
[204]	valid_0's binary_logloss: 0.0536994
[205]	valid_0's binary_logloss: 0.0536903
[206]	valid_0's binary_logloss: 0.0536837
[207]	valid_0's binary_logloss: 0.0536708
[208]	valid_0's binary_logloss: 0.053659
[209]	valid_0's binary_logloss: 0.0536489
[210]	valid_0's binary_logloss: 0.0536366
[211]	valid_0's binary_logloss: 0.05363
[212]	valid_0's binary_logloss: 0.0536221
[213]	valid_0's binary_logloss: 0.053613
[214]	valid_0's binary_logloss: 0.0536077
[215]	valid_0's binary_logloss: 0.0536021
[216]	valid_0's binary_logloss: 0.0535901
[217]	valid_0's binary_logloss: 0.053582
[218]	valid_0's binary_logloss: 0.0535741
[219]	valid_0's binary_logloss: 0.0535648
[220]	valid_0's binary_logloss: 0.0535469
[221]	valid_0's binary_logloss: 0.0535391
[222]	valid_0's binary_logloss: 0.0535334
[223]	valid_0's binary_logloss: 0.0535284
[224]	valid_0's binary_logloss: 0.053521
